In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
from openpyxl import load_workbook
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestRegressor
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import multiprocessing
import os
import time
import gloce as gc
from osgeo import gdal
from osgeo import gdalconst
from mpl_toolkits.basemap import Basemap
from glob import glob
from math import ceil
import seaborn as sns
from scipy import stats
from scipy.stats import gaussian_kde
#return im_data, im_width, im_height, im_geotrans, im_proj
def read_img(filename):
    dt = gdal.Open(filename)
    im_width = dt.RasterXSize
    im_height = dt.RasterYSize
    im_bands = dt.RasterCount
    im_geotrans = dt.GetGeoTransform()
    im_proj = dt.GetProjection()
    im_data = dt.ReadAsArray(0,0,im_width,im_height)
    return im_data, im_width, im_height, im_geotrans, im_proj
def write_img(filename, im_proj, im_geotrans, im_data):
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32
    if len(im_data.shape) == 3:
        im_bands, im_height, im_width = im_data.shape
    else:
        im_bands, (im_height, im_width) = 1, im_data.shape
    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(filename, im_width, im_height, im_bands, datatype)
    dataset.SetGeoTransform(im_geotrans)
    dataset.SetProjection(im_proj)
    if im_bands == 1:
        dataset.GetRasterBand(1).WriteArray(im_data)
    else:
        for i in range(im_bands):
            dataset.GetRasterBand(i + 1).WriteArray(im_data[i])
spei_path='//weili/User2/mxsun/CESS_230818/SPEI_highRes/'
sm_path='//weili/User2/mxsun/CESS_230818/SPEI_highRes/RF/'
soil_path='//weili/User2/mxsun/CESS_230818/SPEI_highRes/RF/'
drv_path='//weili/User2/mxsun/CESS_230818/Pattern_drive/'

In [2]:
DEAA=read_img(spei_path+'SPEI_deaa001_240504.tif')[0]
DAA=read_img(spei_path+'SPEI_daa001_240504.tif')[0]
DEAA[DEAA==0]=np.nan
DAA[DAA==0]=np.nan

In [4]:
pathin='//weili/User2/mxsun/CESS_230818/elephand_disturb/update230524/'
csif12=read_img(pathin+'CSIF_yrMean_2012_afr001.tif')[0]
csif13=read_img(pathin+'CSIF_yrMean_2013_afr001.tif')[0]
csif14=read_img(pathin+'CSIF_yrMean_2014_afr001.tif')[0]
csif15=read_img(pathin+'CSIF_yrMean_2015_afr001.tif')[0]
csif16=read_img(pathin+'CSIF_yrMean_2016_afr001.tif')[0]

ndvi12=read_img(pathin+'NDVI_yrMean_2012_afr001.tif')[0]
ndvi13=read_img(pathin+'NDVI_yrMean_2013_afr001.tif')[0]
ndvi14=read_img(pathin+'NDVI_yrMean_2014_afr001.tif')[0]
ndvi15=read_img(pathin+'NDVI_yrMean_2015_afr001.tif')[0]
ndvi16=read_img(pathin+'NDVI_yrMean_2016_afr001.tif')[0]
#求Resistance
ndvi12[ndvi12<0]=np.nan
ndvi13[ndvi13<0]=np.nan
ndvi14[ndvi14<0]=np.nan
ndvi15[ndvi15<0]=np.nan
ndvi16[ndvi16<0]=np.nan

csif12[csif12<0]=np.nan
csif13[csif13<0]=np.nan
csif14[csif14<0]=np.nan
csif15[csif15<0]=np.nan
csif16[csif16<0]=np.nan

In [5]:
ndvi_dr=(ndvi15+ndvi16)/2
ndvi_predr=(ndvi12+ndvi13+ndvi14)/3
cisf_dr=(csif15+csif16)/2
cisf_predr=(csif12+csif13+csif14)/3

## slide window

In [6]:
dem=read_img(spei_path+'gtopoDEM_afr_001.tif')[0]
slope=read_img(spei_path+'gtopoSlope_afr_001.tif')[0]
print('dem',dem.min(),dem.max())
print('slope',slope.min(),slope.max())
dem=dem.astype(np.float32)
dem[dem==-9999]=np.nan
slope[slope==0]=np.nan

dem -9999 5438
slope -3.402823e+38 79.12291


In [7]:
#forest_mask=read_img(spei_path+'Africa_forest_area_240319.tif')[0]
#forest_mask[forest_mask==0]=np.nan
hfp = read_img(spei_path+'hfp2018_Afirca_001_230525.tif')[0]
hfp[hfp<=15]=1
hfp[hfp>1]=np.nan
mask=hfp#*protec#forest_mask
mask[~np.isnan(mask)]=1
np.nanmin(mask),np.nanmax(mask)

(1.0, 1.0)

In [12]:
"""
supply paired sites methods. 
Be notes the  suppled detail locates should be consistence with all factors.
"""
# current shape:(6500,7900)
#0.01deg to 0.25 deg window size=625 pixels
#6500/25,7900/25...(260,316)
def slide_window_mean(s,dist,res):
    #s: window size
    #dist: disturb array
    #res: resistance and resilience array
    mean_arr=np.zeros((260,316))
    weight_arr=np.zeros((260,316))
    loc=np.zeros((260,316)) #record the location of supplied sites
    sl=np.ones((25,25))
    for i in range(260):
        #print("sliding", i, "line...")
        for j in range(316):
            bloc_dist=dist[i*s:(i+1)*s,j*s:(j+1)*s]
            res_dist=res[i*s:(i+1)*s,j*s:(j+1)*s]
            res_mask=np.multiply(bloc_dist,res_dist)#product restance array controled by window
            count=np.count_nonzero(~np.isnan(res_mask))
            weight=count/625#calculate the weight
            if weight>0.05:
                weight_arr[i,j]=weight
                dt=np.where(res_mask!=np.nan,res_mask,np.nan)
                mean_arr[i,j]=np.nanmean(res_mask)#calculate the mean value
                loc[i,j]=9
            else:
                bloc_dist_a=[] #should be 8 nums [-1,0],[0,-1],[-1,-1],[1,0],[0,1],[1,1],[-1,1],[1,-1]
                res_dist_a=[] #should be 8 nums
                res_mask_pool=[]
                weight_pool=[]
                for m in [-1,1]:
                    bloc_dist_a.append(dist[(i+m)*s:((i+m)+1)*s,j*s:(j+1)*s])
                    bloc_dist_a.append(dist[i*s:(i+1)*s,(j+m)*s:((j+m)+1)*s])
                    bloc_dist_a.append(dist[(i+m)*s:((i+m)+1)*s,(j+m)*s:((j+m)+1)*s])
                    res_dist_a.append(res[(i+m)*s:((i+m)+1)*s,j*s:(j+1)*s])
                    res_dist_a.append(res[i*s:(i+1)*s,(j+m)*s:((j+m)+1)*s])
                    res_dist_a.append(res[(i+m)*s:((i+m)+1)*s,(j+m)*s:((j+m)+1)*s])
                bloc_dist_a.append(dist[(i-1)*s:((i-1)+1)*s,(j+1)*s:((j+1)+1)*s])
                bloc_dist_a.append(dist[(i+1)*s:((i+1)+1)*s,(j-1)*s:((j-1)+1)*s])
                res_dist_a.append(res[(i-1)*s:((i-1)+1)*s,(j+1)*s:((j+1)+1)*s])
                res_dist_a.append(res[(i+1)*s:((i+1)+1)*s,(j-1)*s:((j-1)+1)*s])
                #the smooth order of a 9 value array is [-1,0],[0,-1],[-1,-1],[1,0],[0,1],[1,1],[-1,1],[1,-1]
                for n in range(len(bloc_dist_a)): #the length is 8
                    res_mask_pool.append(np.multiply(bloc_dist_a[n],res_dist_a[n]))
                    weight_pool.append(np.count_nonzero(~np.isnan(res_mask_pool[n]))/625)#calculate the weight
                key=np.argmax(weight_pool) 
                # Key used to loc the supplied direction, the key-value consistience with [-1,0],[0,-1],[-1,-1],[1,0],[0,1],[1,1],[-1,1],[1,-1]
                if weight_pool[key] > 0.05:
                    weight_arr[i,j]=weight_pool[key]
                    dt1=np.where(res_mask_pool[key]!=np.nan,res_mask_pool[key],np.nan)
                    mean_arr[i,j]=np.nanmean(dt1)#calculate the mean value
                    loc[i,j]=key+1
                else:
                    mean_arr[i,j]=np.nan#calculate the mean value
                    loc[i,j]=np.nan
    return mean_arr

dist_pool=[DEAA*mask,DAA*mask]
res_pool=[ndvi_dr,cisf_dr,dem,slope] #protect need conver to fraction
dist_name=['DEAA','DAA']
res_name=['ndvi_dr','cisf_dr','dem','slope']
# protect 最后直接加上去
aa=[]
aa_name=[]
for i in range(2):
    for j in range(4):
        el=[25,dist_pool[i],res_pool[j]]
        aa.append(el)
        aa_name.append('Vegetation_{}_{}_0.25deg_240617.npy'.format(dist_name[i],res_name[j]))
aa_name

['Vegetation_DEAA_ndvi_dr_0.25deg_240617.npy',
 'Vegetation_DEAA_cisf_dr_0.25deg_240617.npy',
 'Vegetation_DEAA_dem_0.25deg_240617.npy',
 'Vegetation_DEAA_slope_0.25deg_240617.npy',
 'Vegetation_DAA_ndvi_dr_0.25deg_240617.npy',
 'Vegetation_DAA_cisf_dr_0.25deg_240617.npy',
 'Vegetation_DAA_dem_0.25deg_240617.npy',
 'Vegetation_DAA_slope_0.25deg_240617.npy']

In [13]:
for i in range(8):
    print('执行任务%s (%s)...' % (i, os.getpid()))
    kk=slide_window_mean(aa[i][0],aa[i][1],aa[i][2])
    np.save(spei_path+aa_name[i],kk)

执行任务0 (13972)...
执行任务1 (13972)...
执行任务2 (13972)...
执行任务3 (13972)...
执行任务4 (13972)...
执行任务5 (13972)...
执行任务6 (13972)...
执行任务7 (13972)...


## 成对样点求Δ

In [14]:
######################mask DEM and Slope############################
# check the Δ dem and Δ slope distribution
dem1=np.load(spei_path+'Vegetation_DEAA_dem_0.25deg_240617.npy',allow_pickle=True)
dem2=np.load(spei_path+'Vegetation_DAA_dem_0.25deg_240617.npy',allow_pickle=True)
dem_differ=dem1-dem2

slo1=np.load(spei_path+'Vegetation_DEAA_slope_0.25deg_240617.npy',allow_pickle=True)
slo2=np.load(spei_path+'Vegetation_DAA_slope_0.25deg_240617.npy',allow_pickle=True)
slo_differ=slo1-slo2

dem_differ[dem_differ<-200]=np.nan
dem_differ[dem_differ>200]=np.nan
dem_differ[~np.isnan(dem_differ)]=1
slo_differ[slo_differ<-10]=np.nan
slo_differ[slo_differ>10]=np.nan
slo_differ[~np.isnan(slo_differ)]=1
# 每一层数据位置对应,建立mask
dd_mask=np.ones((260,316))*dem_differ*slo_differ

In [18]:
res_name=['ndvi_dr','cisf_dr']
delta=[]
for i in range(2):
    f1=np.load(spei_path+'Vegetation_DEAA_{}_0.25deg_240617.npy'.format(res_name[i]),allow_pickle=True)*dd_mask
    f2=np.load(spei_path+'Vegetation_DAA_{}_0.25deg_240617.npy'.format(res_name[i]),allow_pickle=True)*dd_mask
    ff=(f1-f2)/f2
    delta.append(ff)
np.nanmean(delta[0]),np.nanmean(delta[1])#,relative_delta

(0.027545300293898345, 0.04527886852060257)

In [20]:
def kill_nan(dt):
    a=dt.ravel()
    a=a[~np.isnan(a)]
    a=list(a)
    return a
se=[]
for i in range(2):
    se.append(stats.sem(kill_nan(delta[i])))
se

[0.0021454265885783824, 0.004156559485303399]